### Recommendations with MovieTweetings: Most Popular Recommendation

Now that you have created the necessary columns we will be using throughout the rest of the lesson on creating recommendations, let's get started with the first of our recommendations.

To get started, read in the libraries and the two datasets you will be using throughout the lesson using the code below.


In [139]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tests as t

%matplotlib inline

# Read in the datasets
movies = pd.read_csv('movies_clean.csv')
reviews = pd.read_csv('reviews_clean.csv')
del movies['Unnamed: 0']
del reviews['Unnamed: 0']

In [136]:
movies.head()

,movie_id,movie,genre,date,1800's,1900's,2000's,History,News,Horror,...,Fantasy,Romance,Game-Show,Action,Documentary,Animation,Comedy,Short,Western,Thriller
11954,337926,"Chatô, O Rei do Brasil (2015)",Biography,2015,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
12393,369610,Jurassic World (2015),Action|Adventure|Sci-Fi,2015,0,0,1,0,0,0,...,0,0,0,1,0,0,0,0,0,0
13156,420293,The Stanford Prison Experiment (2015),Biography|Drama|History,2015,0,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
13817,462335,High-Rise (2015),Action|Drama|Sci-Fi,2015,0,0,1,0,0,0,...,0,0,0,1,0,0,0,0,0,0
13935,470752,Ex Machina (2015),Drama|Mystery|Sci-Fi,2015,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [138]:
reviews.head()

,user_id,movie_id,rating,timestamp,date,month_1,month_2,month_3,month_4,month_5,...,month_9,month_10,month_11,month_12,year_2013,year_2014,year_2015,year_2016,year_2017,year_2018
0,1,68646,10,1381620027,2013-10-12 23:20:27,0,0,0,0,0,...,0,1,0,0,1,0,0,0,0,0
1,1,113277,10,1379466669,2013-09-18 01:11:09,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
2,2,422720,8,1412178746,2014-10-01 15:52:26,0,0,0,0,0,...,0,1,0,0,0,1,0,0,0,0
3,2,454876,8,1394818630,2014-03-14 17:37:10,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
4,2,790636,7,1389963947,2014-01-17 13:05:47,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0


#### 1. How To Find The Most Popular Movies

For this notebook, we have a single task.  The task is that no matter the user, we need to provide a list of the recommendations based on simply the most popular items.

For this task, we will consider what is "most popular" based on the following criteria:

* A movie with the highest average rating is considered best
* With ties, movies that have more ratings are better
* A movie must have a minimum of 5 ratings to be considered among the best movies
* If movies are tied in their average rating and number of ratings, the ranking is determined by the movie that is the most recent rating

With these criteria, the goal for this notebook is to take a **user_id** and provide back the **n_top** recommendations.  Use the function below as the scaffolding that will be used for all the future recommendations as well.

In [7]:
reviews.movie_id.nunique()

31245

In [39]:
avg_rating = pd.Series(reviews.groupby(by=['movie_id']).rating.mean()).rename('avg_rating')

In [40]:
rating_count = pd.Series(reviews.groupby(by=['movie_id']).rating.count()).rename('rating_count')

In [41]:
rating_time_max = pd.Series(reviews.groupby(by=['movie_id']).date.max()).rename('rating_time_max')

In [42]:
df = pd.DataFrame(pd.concat([avg_rating,rating_count,rating_time_max], axis=1))

Index(['avg_rating', 'rating_count', 'rating_time_max'], dtype='object')

In [45]:
df.head(10)

,avg_rating,rating_count,rating_time_max
movie_id,,,
8,5.000000,1,2014-04-08 18:20:11
10,10.000000,1,2014-10-09 18:15:53
12,10.000000,1,2015-08-10 23:16:19
25,8.000000,1,2017-02-27 10:04:59
91,6.000000,1,2013-11-23 18:59:55
417,8.368421,19,2018-06-26 21:08:46
439,6.750000,4,2018-06-24 02:59:09
443,8.000000,1,2016-10-23 21:08:32
628,4.500000,2,2013-11-29 12:55:28


In [47]:
df_5 = df[df.rating_count >= 5]

In [70]:
top_20_df = df_5.sort_values(by=['avg_rating', 'rating_count', 'rating_time_max'], ascending=[False,False,False]).head(20)

In [71]:
top_20_df = top_20_df.reset_index()

In [72]:
list(top_20_df.merge(movies,how='inner',on='movie_id').movie.values)

['MSG 2 the Messenger (2015)',
 'Avengers: Age of Ultron Parody (2015)',
 'Sorry to Bother You (2018)',
 'Selam (2013)',
 "Quiet Riot: Well Now You're Here, There's No Way Back (2014)",
 'Crawl Bitch Crawl (2012)',
 'Make Like a Dog (2015)',
 'Pandorica (2016)',
 'Third Contact (2011)',
 'Romeo Juliet (2009)',
 'Be Somebody (2016)',
 'Birlesen Gonuller (2014)',
 'Agnelli (2017)',
 'Sátántangó (1994)',
 'Shijie (2004)',
 'Foster (2011)',
 'CM101MMXI Fundamentals (2013)',
 'Akahige (1965)',
 'Crystal Lake Memories: The Complete History of Friday the 13th (2013)',
 'Körkarlen (1921)']

In [92]:
def popular_recommendations(user_id, n_top):
    '''
    INPUT:
    user_id - the user_id of the individual you are making recommendations for
    n_top - an integer of the number recommendations you want back
    OUTPUT:
    top_movies - a list of the n_top recommended movies by movie title in order best to worst
    '''

    avg_rating = pd.Series(reviews.groupby(by=['movie_id']).rating.mean()).rename('avg_rating')
    rating_count = pd.Series(reviews.groupby(by=['movie_id']).rating.count()).rename('rating_count')
    rating_time_max = pd.Series(reviews.groupby(by=['movie_id']).date.max()).rename('rating_time_max')
    df = pd.DataFrame(pd.concat([avg_rating,rating_count,rating_time_max], axis=1))
    df_5 = df[df.rating_count >= 5]
    top_n_df = df_5.sort_values(by=['avg_rating', 'rating_count', 'rating_time_max'], ascending=[False,False,False]).head(n_top)
    top_n_df = top_n_df.reset_index()
    top_movies = list(top_n_df.merge(movies,how='inner',on='movie_id').movie.values)
    
    return top_movies # a list of the n_top movies as recommended

Usint the three criteria above, you should be able to put together the above function.  If you feel confident in your solution, check the results of your function against our solution. On the next page, you can see a walkthrough and you can of course get the solution by looking at the solution notebook available in this workspace.  

In [68]:
# Put your solutions for each of the cases here

# Top 20 movies recommended for id 1

recs_20_for_1 = popular_recommendations(1, 20)  # Your solution list here

# Top 5 movies recommended for id 53968
recs_5_for_53968 = popular_recommendations(53968, 5) # Your solution list here

# Top 100 movies recommended for id 70000
recs_100_for_70000 = popular_recommendations(70000, 100) # Your solution list here

# Top 35 movies recommended for id 43
recs_35_for_43 = popular_recommendations(43, 35) # Your solution list here



In [69]:
### You Should Not Need To Modify Anything In This Cell
ranked_movies = t.create_ranked_df(movies, reviews) # only run this once - it is not fast

# check 1 
assert t.popular_recommendations('1', 20, ranked_movies) == recs_20_for_1,  "The first check failed..."
# check 2
assert t.popular_recommendations('53968', 5, ranked_movies) == recs_5_for_53968,  "The second check failed..."
# check 3
assert t.popular_recommendations('70000', 100, ranked_movies) == recs_100_for_70000,  "The third check failed..."
# check 4
assert t.popular_recommendations('43', 35, ranked_movies) == recs_35_for_43,  "The fourth check failed..."

print("If you got here, looks like you are good to go!  Nice job!")

If you got here, looks like you are good to go!  Nice job!


**Notice:** This wasn't the only way we could have determined the "top rated" movies.  You can imagine that in keeping track of trending news or trending social events, **you would likely want to create a time window from the current time, and then pull the articles in the most recent time frame.  There are always going to be some subjective decisions to be made.**  

If you find that no one is paying any attention to your most popular recommendations, then it might be time to find a new way to recommend, which is what the next parts of the lesson should prepare us to do!


### Part II: Adding Filters

Now that you have created a function to give back the **n_top** movies, let's make it a bit more robust.  Add arguments that will act as filters for the movie **year** and **genre**.  

Use the cells below to adjust your existing function to allow for **year** and **genre** arguments as **lists** of **strings**.  Then your ending results are filtered to only movies within the lists of provided years and genres (as `or` conditions).  If no list is provided, there should be no filter applied.

You can adjust other necessary inputs as necessary to retrieve the final results you are looking for!


In [130]:
# movies[movies.date.isin([2015])]

In [131]:
genre_ = 'Documentary'
# movies[movies['genre'].str.contains(genre_, na=False)] 

In [101]:
# movies[movies[genre_]>0] 

In [132]:
# years = [2015]
# movies[movies['date'].isin(years)]

In [149]:
def popular_recommendations_filtered(user_id, n_top, years = None, genres = None):
    '''
    INPUT:
    user_id - the user_id of the individual you are making recommendations for
    n_top - an integer of the number recommendations you want back
    year - list of filtering year
    genres -list of filtering genres
    OUTPUT:
    top_movies - a list of the n_top recommended movies by movie title in order best to worst
    '''
    global movies
    global reviews
    
    movies_mod = movies
    if(type(years) == list):
        movies_mod = movies_mod[movies_mod['date'].isin(years)]  # use try
    else:
        movies_mod = movies_mod
        
    
    if(type(genres) == list):
        df_list = []
        for i, genre in enumerate(genres):
            df_list.append(movies_mod[movies_mod[genre]>0])     
        movies_mod = pd.concat(df_list)
    print('before', movies_mod.shape[0])
    movies_mod = movies_mod.drop_duplicates() #since is possible to get short|documetry added twice
    print('after', movies_mod.shape[0])
    
#     Another way BETTER
#     if years is not None:
#         movies_mod = movies_mod[movies_mod['date'].isin(years)]

#     if genres is not None:
#         num_genre_match = movies_mod[genres].sum(axis=1)  # atleast one genre is 1
#         movies_mod = movies_mod.loc[num_genre_match > 0, :]        
        
    avg_rating = pd.Series(reviews.groupby(by=['movie_id']).rating.mean()).rename('avg_rating')
    rating_count = pd.Series(reviews.groupby(by=['movie_id']).rating.count()).rename('rating_count')
    rating_time_max = pd.Series(reviews.groupby(by=['movie_id']).date.max()).rename('rating_time_max')
    df = pd.DataFrame(pd.concat([avg_rating,rating_count,rating_time_max], axis=1))
    df_5 = df[df.rating_count >= 5]
    df_prelim = df_5.sort_values(by=['avg_rating', 'rating_count', 'rating_time_max'], ascending=[False,False,False])
    df_prelim = df_prelim.reset_index()
    df_prelim_merge = df_prelim.merge(movies_mod,how='inner',on='movie_id')
    top_movies = list(df_prelim_merge.head(n_top).movie.values)
    

    
    return top_movies # a list of the n_top movies as recommended

In [150]:
popular_recommendations_filtered(1, 20, years=['2015', '2016', '2017', '2018'], genres=['History','Documentary','Short', 'Drama'])

before 3482
after 3205


['MSG 2 the Messenger (2015)',
 'Avengers: Age of Ultron Parody (2015)',
 'Make Like a Dog (2015)',
 'Be Somebody (2016)',
 'Agnelli (2017)',
 'Kirik Party (2016)',
 'Chasing Asylum (2016)',
 'No Stone Unturned (2017)',
 'Dag II (2016)',
 "Hillary's America: The Secret History of the Democratic Party (2016)",
 'Chasing Coral (2017)',
 'Terra (2015)',
 'Ayla: The Daughter of War (2017)',
 'Hitting the Apex (2015)',
 'Colombia magia salvaje (2015)',
 'I Believe in Miracles (2015)',
 'Wild Wild Country (2018)',
 'Bajrangi Bhaijaan (2015)',
 'Dangal (2016)',
 'Inner Workings (2016)']

In [141]:
popular_recommendations_filtered(1, 20, years=None)

['MSG 2 the Messenger (2015)',
 'Avengers: Age of Ultron Parody (2015)',
 'Sorry to Bother You (2018)',
 'Selam (2013)',
 "Quiet Riot: Well Now You're Here, There's No Way Back (2014)",
 'Crawl Bitch Crawl (2012)',
 'Make Like a Dog (2015)',
 'Pandorica (2016)',
 'Third Contact (2011)',
 'Romeo Juliet (2009)',
 'Be Somebody (2016)',
 'Birlesen Gonuller (2014)',
 'Agnelli (2017)',
 'Sátántangó (1994)',
 'Shijie (2004)',
 'Foster (2011)',
 'CM101MMXI Fundamentals (2013)',
 'Akahige (1965)',
 'Crystal Lake Memories: The Complete History of Friday the 13th (2013)',
 'Körkarlen (1921)']

In [142]:
popular_recommendations_filtered(1, 20, years=[2018])

['Sorry to Bother You (2018)',
 'Wild Wild Country (2018)',
 'Ricky Gervais: Humanity (2018)',
 'Avengers: Infinity War (2018)',
 'Hadi Be Oglum (2018)',
 'Incredibles 2 (2018)',
 'Ahlat Agaci (2018)',
 'Deadpool 2 (2018)',
 'Kelebekler (2018)',
 'Isle of Dogs (2018)',
 'Love, Simon (2018)',
 'A Quiet Place (2018)',
 'Upgrade (2018)',
 'The Tale (2018)',
 'Ready Player One (2018)',
 'Padman (2018)',
 'Hichki (2018)',
 'Black Panther (2018)',
 'Game Night (2018)',
 'Set It Up (2018)']

In [148]:
popular_recommendations_filtered(1, 20, years=[2017,2018], genres=['Documentary'])

['Agnelli (2017)',
 'No Stone Unturned (2017)',
 'Chasing Coral (2017)',
 'Wild Wild Country (2018)',
 'The Farthest (2017)',
 'Ricky Gervais: Humanity (2018)',
 'What the Health (2017)',
 'Icarus (2017)',
 'LA 92 (2017)',
 'Gaga: Five Foot Two (2017)',
 'I Am Heath Ledger (2017)',
 'Visages villages (2017)',
 'Get Me Roger Stone (2017)',
 'Mommy Dead and Dearest (2017)',
 'Becoming Warren Buffett (2017)',
 'One of Us (2017)',
 'Chris Brown: Welcome to My Life (2017)',
 'Conor McGregor: Notorious (2017)',
 '78/52 (2017)',
 'City of Ghosts (2017)']

In [147]:
popular_recommendations_filtered(1, 20, years=['2015', '2016', '2017', '2018'], genres=['History'])

['Ayla: The Daughter of War (2017)',
 'I Believe in Miracles (2015)',
 'The Farthest (2017)',
 'Sado (2015)',
 'Hatred (2016)',
 'Kincsem (2017)',
 'Nise - O Coração da Loucura (2015)',
 'LA 92 (2017)',
 'Straight Outta Compton (2015)',
 'Manjhi: The Mountain Man (2015)',
 'Only the Dead (2015)',
 'Spotlight (2015)',
 'Under sandet (2015)',
 'Airlift (2016)',
 'Dunkirk (2017)',
 'Taeksi Woonjunsa (2017)',
 'The Battleship Island (2017)',
 'Darkest Hour (2017)',
 'Best of Enemies (2015)',
 'The Ghazi Attack (2017)']


Try writing a few tests against the test function in our test function.  Below returns the top 20 movies for user 1 based on the specified year and genre filters.  Does yours return the same? 

```
t.popular_recs_filtered('1', 20, ranked_movies, years=['2015', '2016', '2017', '2018'], genres=['History'])
```

In [146]:
t.popular_recs_filtered('1', 20, ranked_movies, years=['2015', '2016', '2017', '2018'], genres=['History'])

['Ayla: The Daughter of War (2017)',
 'I Believe in Miracles (2015)',
 'The Farthest (2017)',
 'Sado (2015)',
 'Hatred (2016)',
 'Kincsem (2017)',
 'Nise - O Coração da Loucura (2015)',
 'LA 92 (2017)',
 'Straight Outta Compton (2015)',
 'Manjhi: The Mountain Man (2015)',
 'Only the Dead (2015)',
 'Spotlight (2015)',
 'Under sandet (2015)',
 'Airlift (2016)',
 'Dunkirk (2017)',
 'Taeksi Woonjunsa (2017)',
 'The Battleship Island (2017)',
 'Darkest Hour (2017)',
 'Best of Enemies (2015)',
 'The Ghazi Attack (2017)']